In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
working_dir = "../input"
# Any results you write to the current directory are saved as output.
import pandas as pd
import numpy as np
import json
import gensim

from keras import optimizers
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.layers import Dense, Activation
import pickle as pkl
from sklearn.preprocessing import OneHotEncoder



['test_images', '.DS_Store', 'train_sentiment', 'move.py', 'train', 'labels.pkl', 'train_images']


Using TensorFlow backend.


In [2]:
def load_csv(path):
    csv_dataset = pd.read_csv(path,index_col="PetID")
    return csv_dataset
csv_dataset = load_csv(working_dir+"/train/"+"train.csv")
csv_dataset.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PhotoAmt,AdoptionSpeed
PetID,,,,,,,,,,,,,,,,,,,,,
86e1089a3,2,Nibble,3,299,0,1,1,7,0,1,...,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,1.0,2
6296e909a,2,No Name Yet,1,265,0,1,1,2,0,2,...,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,2.0,0
3422e4906,1,Brisco,1,307,0,1,2,7,0,2,...,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,7.0,3
5842f1ff5,1,Miko,4,307,0,2,1,2,0,2,...,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",8.0,2
850a43f90,1,Hunter,1,307,0,1,1,0,0,2,...,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,3.0,2


In [3]:
def read_input(input_dict):
    documents = []
    for key, line in input_dict.items():
        tokens = gensim.utils.simple_preprocess(str(line))
        documents.append(tokens)
        discriptions[key] = tokens
    return documents, discriptions

labels = csv_dataset["AdoptionSpeed"].to_dict()
discriptions = csv_dataset["Description"].to_dict()
documents, discriptions = read_input(discriptions)


In [5]:
discriptions

{'86e1089a3': ['nibble',
  'is',
  'month',
  'old',
  'ball',
  'of',
  'cuteness',
  'he',
  'is',
  'energetic',
  'and',
  'playful',
  'rescued',
  'couple',
  'of',
  'cats',
  'few',
  'months',
  'ago',
  'but',
  'could',
  'not',
  'get',
  'them',
  'neutered',
  'in',
  'time',
  'as',
  'the',
  'clinic',
  'was',
  'fully',
  'scheduled',
  'the',
  'result',
  'was',
  'this',
  'little',
  'kitty',
  'do',
  'not',
  'have',
  'enough',
  'space',
  'and',
  'funds',
  'to',
  'care',
  'for',
  'more',
  'cats',
  'in',
  'my',
  'household',
  'looking',
  'for',
  'responsible',
  'people',
  'to',
  'take',
  'over',
  'nibble',
  'care'],
 '6296e909a': ['just',
  'found',
  'it',
  'alone',
  'yesterday',
  'near',
  'my',
  'apartment',
  'it',
  'was',
  'shaking',
  'so',
  'had',
  'to',
  'bring',
  'it',
  'home',
  'to',
  'provide',
  'temporary',
  'care'],
 '3422e4906': ['their',
  'pregnant',
  'mother',
  'was',
  'dumped',
  'by',
  'her',
  'irrespons

In [6]:
model = gensim.models.Word2Vec(
        documents,
        size=150,
        window=10,
        min_count=2,
        workers=10
)

model.train(documents, total_examples=len(documents), epochs=10)

(6821843, 8884670)

In [7]:
model.save('kaggle_information.lexicon')

In [12]:
model.wv.most_similar([""])

[('puppy', 0.5671311020851135),
 ('dogs', 0.5563753247261047),
 ('pup', 0.5359048247337341),
 ('security', 0.5335999131202698),
 ('pet', 0.44463059306144714),
 ('doggie', 0.4339735805988312),
 ('alerting', 0.43365463614463806),
 ('poodle', 0.40949976444244385),
 ('beagle', 0.40798434615135193),
 ('guard', 0.39280518889427185)]

In [86]:
model.wv["cat"].shape

(150,)

In [90]:
pretrained_weights = model.wv.vectors
vocab_size, emdedding_size = pretrained_weights.shape

def word2idx(word):
    try:
        return model.wv.vocab[word].index
    except:
        return 0
def idx2word(idx):
    return model.wv.index2word[idx]

In [107]:
max_sentence_len = 128

In [120]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, 
                    weights=[pretrained_weights]))
lstm_model.add(LSTM(max_sentence_len,return_sequences=True))
lstm_model.add(Dropout(0.5))
lstm_model.add(LSTM(max_sentence_len))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(5))
lstm_model.add(Activation('softmax'))
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [121]:
# max_sentence_len = max(len(s) for s in documents)
print(max_sentence_len)

train_x = np.zeros([len(documents), max_sentence_len], dtype=np.int32)

for i, sentence in enumerate(documents):
    for t, word in enumerate(sentence):
        if t >= max_sentence_len:
            break
        train_x[i, t] = word2idx(word)

128


In [122]:
target = csv_dataset["AdoptionSpeed"]
labels_encoder = OneHotEncoder()
one_hot_labels = labels_encoder.fit_transform(target.values.reshape((target.shape[0],1)))
one_hot_labels.shape

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(14993, 5)

In [123]:
X_train, X_test, y_train, y_test = train_test_split(
    train_x, one_hot_labels, test_size=0.2, random_state=0)

In [124]:
X_train.shape,X_test.shape

((11994, 128), (2999, 128))

In [125]:
lstm_model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs = 50, batch_size = 256, shuffle=True)

Train on 11994 samples, validate on 2999 samples
Epoch 1/50
11994/11994 [==============================] - 54s 5ms/step - loss: 1.5011 - acc: 0.2671 - val_loss: 1.4673 - val_acc: 0.2871
Epoch 2/50
11994/11994 [==============================] - 51s 4ms/step - loss: 1.4770 - acc: 0.2788 - val_loss: 1.4644 - val_acc: 0.2818
Epoch 3/50
11994/11994 [==============================] - 50s 4ms/step - loss: 1.4733 - acc: 0.2789 - val_loss: 1.4667 - val_acc: 0.2834
Epoch 4/50
11994/11994 [==============================] - 51s 4ms/step - loss: 1.4669 - acc: 0.2884 - val_loss: 1.4620 - val_acc: 0.2831
Epoch 5/50
11994/11994 [==============================] - 51s 4ms/step - loss: 1.4624 - acc: 0.2921 - val_loss: 1.4696 - val_acc: 0.2664
Epoch 6/50
11994/11994 [==============================] - 50s 4ms/step - loss: 1.4544 - acc: 0.2991 - val_loss: 1.4680 - val_acc: 0.2641
Epoch 7/50
11994/11994 [==============================] - 51s 4ms/step - loss: 1.4418 - acc: 0.2999 - val_loss: 1.4763 - val_acc: